In [ ]:
import warnings 
warnings.filterwarnings('ignore')
from modules.Utils.utils import loadFromDB
from concurrent.futures import ProcessPoolExecutor, as_completed
from multiprocess import cpu_count

from modules.Utils.utils import loadFromDB, strategyTester, getFearAndGreedIndicator,computeStochasticLinearRegression,computeEMD
from modules.Utils.indicators import computeSuperTrend, addIndicators, computeLaggingLinearRegression, computeTrixIndicator, generateDatesFeatures

import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler


In [ ]:
currencies = ['BTC','ETH','ADA','AAVE','ALGO','EGLD','ETH','LINK','DOT','SOL','UNI','XRP','AVAX','AXS','NEAR','MATIC']
print(f'Total currencies : {len(currencies)}')
def getCurrencies(symbol:str):
    df_copy = loadFromDB(symbol,'12h')
    
    #df_copy = computeLaggingLinearRegression(df_copy,window=10,filter_ceof=False, derivative=False)
    df_copy = addIndicators(df_copy,derivative=True, b_engulfings=True)
    df_copy = generateDatesFeatures(df_copy)
    #df_copy = getFearAndGreedIndicator(df_copy)
    df_copy = computeTrixIndicator(df_copy, col='Close')
    df_copy['Return'] =df_copy.Close.pct_change()
    df_copy['Target'] = df_copy.Return.shift(-1)
    df_copy.drop(columns=['Timestamp'],inplace=True)
    df_copy.dropna(inplace=True)
    df_copy['Target'] =  df_copy['Target'].apply(lambda x: 1 if x>=0 else 0)
    #df_copy.drop(columns=['Timestamp','B_MLR_coefs_filtered','B_MLR_coefs_filtered_diff','Prev_B_MLR_coefs_filtered_diff'],inplace=True)
    print(f'{symbol} : {df_copy.shape}')
    return df_copy.dropna()

def scaleDfs(df:pd.DataFrame):
    y = df.Target.values.reshape(-1,1)
    X = df.drop(columns=['Target'])
    
    #X_scaled = PCA(n_components=15).fit_transform(StandardScaler().fit_transform(X))
    X_scaled = StandardScaler().fit_transform(X)
    
    return {'X':X_scaled,
            'y':y} 

def main(symbol:str):
    return scaleDfs(getCurrencies(symbol))

with ProcessPoolExecutor(max_workers=cpu_count()) as executor:
    processes = [executor.submit(main, cur) for cur in currencies]

x_n_y = [task.result() for task in as_completed(processes)]

print('Splitting done !')

X = np.vstack(tuple([x['X'] for x in x_n_y]))
y = np.vstack(tuple([x['y'] for x in x_n_y]))
#np.save('./data/X.npy',X)
#np.save('./data/y.npy',y)
print('Saving done !')
X.shape, y.shape

In [ ]:
from pyts.datasets import load_gunpoint
from pyts.image import GramianAngularField, MarkovTransitionField

transformer = GramianAngularField(method='summation')#MarkovTransitionField(n_bins=20,strategy='normal')#
X_new = transformer.transform(X)
#X_new_2 = np.array([x.reshape(15,15,1) for x in X_new ])
X_new.shape

In [ ]:
from matplotlib import pyplot as plt
from time import sleep
from IPython import display

#fig, ax = plt.subplots(1, figsize=(25,8))
for i in X_new:
    plt.imshow(i, cmap='rainbow', origin='lower')
    plt.show()
    display.clear_output(wait=True)
    sleep(1)

In [ ]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten, Conv1D,Conv2D, GRU, InputLayer, RepeatVector, TimeDistributed, MaxPooling2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import regularizers
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.activations import relu
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError, BinaryCrossentropy

In [ ]:
model = Sequential()
model.add(InputLayer((35, 35)))
model.add(LSTM(units = 64, return_sequences = True))
model.add(LSTM(units = 32, return_sequences = True))
for dilation_rate in (1,2,4,8,16,23):
    model.add(Conv1D(filters=32,kernel_size=2,dilation_rate=dilation_rate,strides=1,padding='causal',activation=keras.activations.relu))
model.add(Conv1D(filters=1,kernel_size=2)) 

#model.add(InputLayer((15, 15,1)))
#model.add(Conv2D(filters=32, activation='relu',kernel_size=2)) 
#model.add(MaxPooling2D((2,2)))
#model.add(Conv2D(filters=64, activation='relu',kernel_size=2)) 
#model.add(MaxPooling2D(2))
#model.add(Conv2D(filters=64, activation='relu',kernel_size=1)) 

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(units = 64,activation=keras.activations.relu))
model.add(Dense(units = 1,activation=keras.activations.sigmoid))

model.compile(optimizer = Adam(learning_rate=0.001), loss = 'binary_crossentropy',metrics=['accuracy'])

model.fit(X_new,
          y,
          epochs=20,
          shuffle=True,
          batch_size=128,
          validation_split=0.2,
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                                        verbose=1,
                                                       patience=8,
                                                       min_delta=0.001,
                                                       mode='max',
                                                       restore_best_weights=True)],
          use_multiprocessing=True)

In [ ]:
from sklearn.metrics import plot_confusion_matrix


class estimator:
  _estimator_type = ''
  classes_=[]
  def __init__(self, model, classes):
    self.model = model
    self._estimator_type = 'classifier'
    self.classes_ = classes
  def predict(self, X):
    y_prob= self.model.predict(X)
    y_pred = y_prob.argmax(axis=1)
    return y_pred

classifier = estimator(model, ['0','1'])
plot_confusion_matrix(estimator=classifier, X=X_new_2, y_true=y)

In [ ]:
from pyts.image import GramianAngularField
from matplotlib import pyplot as plt
import os
from multiprocessing import Pool, cpu_count

transformer = MarkovTransitionField(n_bins=8,strategy='normal')#GramianAngularField(image_size=30, method='summation')
X_new = transformer.transform(X)

def convert_img(idx):
    fig = plt.figure()
    ax = plt.subplot(111)
    try:
        fname = f'./imgs/{idx}.png'
        if os.path.exists(fname):
            return
    except:
        return
    ax.imshow(X_new[idx], cmap='rainbow', origin='lower')
    ax.set_title('')
    ax.set_xticks([])
    ax.set_yticks([])
    fig.savefig(fname, bbox_inches='tight',pad_inches=0,transparent=True)
    
p = Pool(cpu_count())
_ = p.map(convert_img, (i for i in range(X_new.shape[0])))